In [3]:
import pandas as pd
import numpy as np

import json
import matplotlib.image as img
import matplotlib.pyplot as plt

In [4]:
# Read Data from Driving Log
driving_log = pd.read_csv('driving_log.csv', index_col=False)
driving_log.columns = ['Center Image', 'Left Image', 'Right Image', 'Steering Angle', 'Throttle', 'Break', 'Speed']

X_train_path, y_train = [], []
for index, row in driving_log.iterrows():
    # Include center image and steering angle.
    X_train_path.append(row['Center Image'])
    y_train.append(row['Steering Angle'])

#Read in image files (will need to change for generator later)
X_train = []
for path in X_train_path:
    X_train.append(img.imread(path))
    
X_train = np.array(X_train)
y_train = np.array(y_train).reshape(-1,1)

# #Display Histogram
# driving_log['Steering Angle'].hist(bins=25)
# plt.title('Steering Angle Distribution')
# plt.xlabel('Streering Angle')
# plt.ylabel('Frequency')
# plt.show()
    

In [5]:
from sklearn.utils import shuffle

X_train, y_train = shuffle(X_train, y_train)

def normalize(image_data):
    """
    Normalize the image data with Min-Max scaling to a range of [0.1, 0.9]
    :param image_data: The image data to be normalized
    :return: Normalized image data
    """
    a = -0.5
    b = 0.5
    min_val = 0
    max_val = 255
    return a + ( ( (image_data - min_val)*(b - a) )/( max_val - min_val ) )

X_normalized = normalize(X_train)
del X_train

MemoryError: 

In [ ]:
# TODO: Build a model
from keras.models import Sequential
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

# Above model with 10 epochs
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(32, 32, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(43))
model.add(Activation('softmax'))


# TODO: Compile and train the model
model.compile('adam', 'categorical_crossentropy', ['accuracy'])
history = model.fit(X_normalized, y_one_hot, nb_epoch=10, validation_split=0.2)

filepath = './models/keras.h5'
model.save(filepath)